# PTST Idealization using axi-symmetric 2D interface layer

## Purpose of the notebook

Verify the axi-symmetric model in an elastic state to ensure that the confinement is
correctly reproduced. This notebook serves as a basis for 3D degradation within
a ligament/bond zone in axi-symmetric models. Examples are
 - PTST
 - prestressing
 - confined bond configuration
 - effect of splitting cracks
 - restrained splitting cracks in fatigue tests

## Implementation remarks

- The notebook now uses `BCSliceI` - to ensure the same order of nodes along an 
interfaces
- Compression is applied using individual `BCDof` instances without integration 
  over the boundary. This should be changed to BCSliceE - which can use the 
  finite element shape functions to integrate over the element domain

# Elastic material model for the ligament

 - check the uniformity of the radial stress 

In [73]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

## FE Axisymmetric discretization

In [67]:
n_x_e = 20
n_inner_y_e = 10
n_outer_y_e = 10
L_x = 100.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
len_notch = 10
n_notch = 2
# geo_transform
# gmesh
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
m_inelastic = MATS3DElastic(E=37000, nu=0.18)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [68]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[n_notch:-n_notch, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[n_notch:-n_notch, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [69]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
#       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [70]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [71]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [60]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [61]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



## Postprocesing

### Access the last displacement step

In [62]:
U_last = m.hist.U_t[-1]

In [63]:
#U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

(array([[-1.09606153e-02, -1.68541575e-02],
        [-1.13161723e-02, -3.65336555e-02],
        [-9.49107992e-03, -2.05594421e-02],
        [-1.07469740e-02, -2.98337601e-02],
        [-8.90442279e-03, -1.97726155e-02],
        [-1.05895966e-02, -2.69007319e-02],
        [-8.37629507e-03, -1.85918671e-02],
        [-1.01037060e-02, -2.49618675e-02],
        [-7.77192498e-03, -1.75356618e-02],
        [-9.32190626e-03, -2.37190153e-02],
        [-7.06248776e-03, -1.67540370e-02],
        [-8.30010274e-03, -2.29528120e-02],
        [-6.25541331e-03, -1.62415953e-02],
        [-7.10747116e-03, -2.25104855e-02],
        [-5.37646084e-03, -1.59581247e-02],
        [-5.80869592e-03, -2.22853621e-02],
        [-4.45992745e-03, -1.58680702e-02],
        [-4.45992745e-03, -2.22168317e-02],
        [-3.54339405e-03, -1.59581247e-02],
        [-3.11115898e-03, -2.22853621e-02],
        [-2.66444159e-03, -1.62415953e-02],
        [-1.81238373e-03, -2.25104855e-02],
        [-1.85736713e-03, -1.675

In [74]:
#xd_middle.map_U_to_field(U_last)

In [65]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# Slide34 model used for a ligament

 - check the correctness of the wrapper interface  

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_matmod.slide.vslide_34_ij import MATS3DSlideStrain, Slide34

## FE Axisymmetric discretization

In [2]:
n_x_e = 1
n_inner_y_e = 1
n_outer_y_e = 1
L_x = 100.0 # [mm]
d_x = L_x / n_x_e
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
len_notch = 10
n_notch = 0
# geo_transform
# gmesh
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0+n_notch*d_x, R_in),
                               coord_max=(L_x-n_notch*d_x, R_in+dR),
                               shape=(n_x_e-2*n_notch, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3, 
     E_N=1, S_N=0.5, c_N = 3, m = 0.05, f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)
m_inelastic = MATS3DSlideStrain(slide_displ=bond_m)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [3]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[n_notch:-n_notch, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[n_notch:-n_notch, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [4]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
#       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [5]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [6]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [7]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [8]:
s.reset()
s.run()

t:	 0.00

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

## Postprocesing

### Access the last displacement step

In [62]:
U_last = m.hist.U_t[-1]

In [63]:
#U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

(array([[-1.09606153e-02, -1.68541575e-02],
        [-1.13161723e-02, -3.65336555e-02],
        [-9.49107992e-03, -2.05594421e-02],
        [-1.07469740e-02, -2.98337601e-02],
        [-8.90442279e-03, -1.97726155e-02],
        [-1.05895966e-02, -2.69007319e-02],
        [-8.37629507e-03, -1.85918671e-02],
        [-1.01037060e-02, -2.49618675e-02],
        [-7.77192498e-03, -1.75356618e-02],
        [-9.32190626e-03, -2.37190153e-02],
        [-7.06248776e-03, -1.67540370e-02],
        [-8.30010274e-03, -2.29528120e-02],
        [-6.25541331e-03, -1.62415953e-02],
        [-7.10747116e-03, -2.25104855e-02],
        [-5.37646084e-03, -1.59581247e-02],
        [-5.80869592e-03, -2.22853621e-02],
        [-4.45992745e-03, -1.58680702e-02],
        [-4.45992745e-03, -2.22168317e-02],
        [-3.54339405e-03, -1.59581247e-02],
        [-3.11115898e-03, -2.22853621e-02],
        [-2.66444159e-03, -1.62415953e-02],
        [-1.81238373e-03, -2.25104855e-02],
        [-1.85736713e-03, -1.675

In [74]:
#xd_middle.map_U_to_field(U_last)

In [65]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()